In [1]:
p = 21888242871839275222246405745257275088548364400416034343698204186575808495617

In [2]:
from tensorflow.keras.layers import Input, Dense
from tensorflow.keras import Model
import numpy as np

In [3]:
inputs = Input(shape=(3,))
x = Dense(2, activation='relu')(inputs)
outputs = Dense(1)(x)
model = Model(inputs, outputs)

In [4]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 3)]               0         
                                                                 
 dense (Dense)               (None, 2)                 8         
                                                                 
 dense_1 (Dense)             (None, 1)                 3         
                                                                 
Total params: 11
Trainable params: 11
Non-trainable params: 0
_________________________________________________________________


In [5]:
model.weights

[<tf.Variable 'dense/kernel:0' shape=(3, 2) dtype=float32, numpy=
 array([[-0.72784126,  0.33670223],
        [-0.7780691 ,  0.62421453],
        [-1.0269804 ,  0.49905217]], dtype=float32)>,
 <tf.Variable 'dense/bias:0' shape=(2,) dtype=float32, numpy=array([0., 0.], dtype=float32)>,
 <tf.Variable 'dense_1/kernel:0' shape=(2, 1) dtype=float32, numpy=
 array([[-0.00755942],
        [ 0.58352613]], dtype=float32)>,
 <tf.Variable 'dense_1/bias:0' shape=(1,) dtype=float32, numpy=array([0.], dtype=float32)>]

In [6]:
X = np.random.rand(1,3)
X

array([[0.5545958 , 0.48267873, 0.46073158]])

In [7]:
y = model.predict(X)
y

1/1 [==============================] - 0s 36ms/step


2023-10-24 01:16:53.952999: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


array([[0.41894716]], dtype=float32)

In [8]:
X_in = [int(x*1e36) for x in X[0]]
Dense32weights = [[int(model.weights[0].numpy()[i][j]*1e36) for j in range(2)] for i in range(3)]
Dense32bias = [int(model.weights[1].numpy()[i]*1e72) for i in range(2)]
Dense21weights = [[int(model.weights[2].numpy()[i][j]*1e36) for j in range(1)] for i in range(2)]
Dense21bias = [int(model.weights[3].numpy()[i]*1e72) for i in range(1)]


In [9]:
def DenseInt(nInputs, nOutputs, n, input, weights, bias):
    Input = [str(input[i] % p) for i in range(nInputs)]
    Weights = [[str(weights[i][j] % p) for j in range(nOutputs)] for i in range(nInputs)]
    Bias = [str(bias[i] % p) for i in range(nOutputs)]
    out = [0 for _ in range(nOutputs)]
    remainder = [None for _ in range(nOutputs)]
    for j in range(nOutputs):
        for i in range(nInputs):
            out[j] += input[i] * weights[i][j]
        out[j] += bias[j]
        remainder[j] = str(out[j] % n)
        out[j] = str(out[j] // n % p)
    return Input, Weights, Bias, out, remainder

In [10]:
X_in, Dense32weights, Dense32bias, Dense32out, Dense32remainder = DenseInt(3, 2, 10**36, X_in, Dense32weights, Dense32bias)
Dense32out, Dense32remainder

(['21888242871839275222246405745257275088547112023011337864689255117657466434208',
  '717957812208694328223024234032338630'],
 ['749712352190261448543726154832412672',
  '264894409627262887048751218981601280'])

In [11]:
ReLUout = [Dense32out[i] if int(Dense32out[i]) < p//2 else 0 for i in range(2)]
Dense21in = [int(ReLUout[i]) for i in range(2)]

In [12]:
_, Dense21weights, Dense21bias, Dense21out, Dense21remainder = DenseInt(2, 1, 10**36, Dense21in, Dense21weights, Dense21bias)
Dense21out, Dense21remainder

(['418947146885730875576316926961913803'],
 ['695849152206718637828300508516843520'])

In [13]:
in_json = {
    "in": X_in,
    "Dense32weights": Dense32weights,
    "Dense32bias": Dense32bias,
    "Dense32out": Dense32out,
    "Dense32remainder": Dense32remainder,
    "ReLUout": ReLUout,
    "Dense21weights": Dense21weights,
    "Dense21bias": Dense21bias,
    "Dense21out": Dense21out,
    "Dense21remainder": Dense21remainder
}

In [14]:
import json

In [15]:
with open("model1_input.json", "w") as f:
    json.dump(in_json, f)